# IMPORTS

In [1]:
# all the relevant imports are done here
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
basedir = 'C:\\Users\\rpetr\\OneDrive\\Desktop\\DISS_CODE\\ms2ldaviz\\ms2ldaviz'
sys.path.append(basedir)
import django
import json
django.setup()
from basicviz.models import Experiment, Alpha, Mass2MotifInstance, FeatureInstance, Feature, Document, Mass2Motif, DocumentMass2Motif, FeatureMass2MotifInstance
import numpy as np
import pylab as plt
import csv
from scipy.special import polygamma as pg
from scipy.special import psi as psi

MEDIA_ROOT is C:\Users\rpetr\OneDrive\Desktop\DISS_CODE\ms2ldaviz\ms2ldaviz\media


# VARIABLES

In [2]:
#there is only one experiment id we care about => 190 (mass binned 005) - it has 500 topics and 27923 words and 2132 docs
experiment_id=190 
experiment = Experiment.objects.get(id=experiment_id)
min_prob_beta = 1e-3
SMALL_NUMBER = 1e-100

# CORPUS (features for 3 documents in experiment 190)

In [3]:
# Words are features. These are done here. All results have been checked versus the database. 
features = Feature.objects.filter(experiment_id=experiment)
experiment_words = []
for f in features:
     if f.id not in experiment_words: 
        experiment_words.append(f.id)

In [5]:
# unique_words is the dictionary: word: index
unique_words = {}
index = 0
for word in experiment_words:
    if word not in unique_words.keys():
        unique_words.update({word:index})
        index+=1

In [7]:
#get the unique documents for our experiment
documents = Document.objects.filter(experiment_id=experiment)
experiment_docs=[]
for d in documents:
    if d.id not in experiment_docs: 
        experiment_docs.append(d.id)

In [9]:
#unique documents is the dictionary -> doc: id 
unique_docs = {}
index = 0 
for doc in experiment_docs: 
    unique_docs.update({doc:index})
    index+=1

In [25]:
#then we get the corresponding words for 3 documents (we do not need all of them)
feature_instances = FeatureInstance.objects.filter(document_id__in=unique_docs.keys(), feature_id__in=unique_words.keys())
doc_word_data = []
for f in feature_instances:
    doc_word_data.append([unique_docs[int(f.document_id)], unique_words[int(f.feature_id)], f.intensity])
#output of the above is doc_id, word_id, word_count

In [32]:
#output a csv for the corpus in order to create a dictionary made up of {document_id:{word_id:intensity}}
doc_word_array = np.array(doc_word_data)
np.savetxt("corpus_data.csv", doc_word_array, delimiter=",", fmt="%s")
np.save("corpus_data",doc_word_array)

In [33]:
#CREATE THE CORPUS - a dictionary where key is document id and value is a dict of the count of words
corpus_dict = {}
with open("corpus_data.csv", 'r') as data_file:
    data = csv.DictReader(data_file, delimiter=",")
    for row in data:
        item = corpus_dict.get(row["doc_id"], dict())
        item[row["word_id"]] = int(row["count"])
        corpus_dict[row["doc_id"]] = item
#Get the corpus dict whenever this is necessary

# UNIQUE TOPICS

In [37]:
#get the unique topics - var name: unique_topics (important for alpha, beta calculations)
mi = Mass2Motif.objects.filter(experiment=experiment)
unique_topics = {}
index=0
for m in mi: 
    unique_topics.update({m.id:index})
    index+=1

In [39]:
unique_topics

{57401: 405,
 57402: 139,
 57403: 140,
 57404: 141,
 57405: 142,
 57406: 143,
 57407: 144,
 57408: 145,
 57409: 146,
 57410: 147,
 57411: 148,
 57412: 149,
 57413: 150,
 57414: 151,
 57415: 152,
 57416: 153,
 57417: 154,
 57418: 155,
 57419: 156,
 57420: 157,
 57421: 406,
 57422: 158,
 57423: 159,
 57424: 160,
 57425: 161,
 57426: 162,
 57427: 163,
 57428: 164,
 57429: 165,
 57430: 166,
 57431: 167,
 57432: 168,
 57433: 169,
 57434: 170,
 57435: 171,
 57436: 172,
 57437: 173,
 57438: 174,
 57439: 175,
 57440: 176,
 57441: 177,
 57442: 178,
 57443: 179,
 57444: 407,
 57445: 180,
 57446: 181,
 57447: 182,
 57448: 183,
 57449: 184,
 57450: 408,
 57451: 6,
 57452: 7,
 57453: 8,
 57454: 9,
 57455: 10,
 57456: 11,
 57457: 12,
 57458: 13,
 57459: 14,
 57460: 15,
 57461: 16,
 57462: 409,
 57463: 17,
 57464: 18,
 57465: 19,
 57466: 20,
 57467: 21,
 57468: 22,
 57469: 55,
 57470: 23,
 57471: 24,
 57472: 25,
 57473: 26,
 57474: 27,
 57475: 28,
 57476: 29,
 57477: 30,
 57478: 31,
 57479: 0,
 57480

# ALPHA

In [40]:
# get the alphas from the database
al = Alpha.objects.filter(mass2motif__experiment=experiment).order_by('mass2motif')
output_data_alpha = []
for a in al:
    output_data_alpha.append([a.mass2motif_id, a.value])

In [41]:
# make alpha into an array and vectorise it
output_arr_alpha = np.array(output_data_alpha)
col_pos = output_arr_alpha[:, 0].astype(int)
values = output_arr_alpha[:, 1]
col_pos = np.vectorize(unique_topics.get)(col_pos)
alpha = np.zeros((1,len(col_pos)))
alpha[0,col_pos] = values
alpha_vector=alpha
#np.savetxt("alpha.csv", alpha_vector, delimiter=",", fmt="%s")

# BETA

In [42]:
# we have unique words and topics, so now we get the unique array
beta_pre_pivot = []
mi = Mass2MotifInstance.objects.filter(mass2motif__experiment=experiment)
for m in mi:
    beta_pre_pivot.append([unique_topics[m.mass2motif_id], unique_words[m.feature_id], m.probability]) 
#position topic, position row, probability above row
#results checked against the database - correct 

In [43]:
#some topics may have 0 words - these have been reincluded 
#creating array from the beta data and subsequently creating a pivot (matrix)
output_arr_beta = np.array(beta_pre_pivot)
K = len(unique_topics)
W = len(unique_words)
pivot_table = np.zeros((K, W)).astype('float')
i = 0
max = len(beta_pre_pivot)

while i<max:
    pivot_table[int(output_arr_beta[i][0]),int(output_arr_beta[i][1])]=output_arr_beta[i][2]
    i+=1

In [44]:
# use this should you need to print
# np.savetxt("temp.csv", pivot_table, delimiter=",")

In [45]:
#normalise the beta pivot table matrix 
pivot_table_normalised = pivot_table
i = 0
while i<500: 
    row = pivot_table_normalised[i, :]
    adjusted_row = row + 1e-8
    normalised_row = adjusted_row / np.sum(adjusted_row)
    np.sum(normalised_row)
    pivot_table_normalised[i, :] = normalised_row
    i+=1
#np.savetxt("beta_matrix.csv", pivot_table_normalised, delimiter=",")

# VISUALISATION

In [48]:
# use this for visualisation if necessary 
# my_dpi=150
# plt.figure(figsize=(2000/my_dpi, 2000/my_dpi), dpi=my_dpi)
# plt.imshow(pivot_table_normalised, aspect="auto")

# GET ORIGINAL THETA(NORM GAMMA)

In [49]:
#get the original theta from the database for subsequent comparison 
# remember theta is jsut normalised gamma
theta = DocumentMass2Motif.objects.filter(document_id__in=experiment_docs)
output_data_theta = []
for t in theta:
    output_data_theta.append([unique_docs[int(t.document_id)], unique_topics[int(t.mass2motif_id)], t.probability])
#checked if it is correct against the database

In [50]:
output_data_theta

[[513, 218, 0.990136521727459],
 [512, 413, 0.0122284803668866],
 [512, 164, 0.984625841740529],
 [3, 446, 0.973540791004048],
 [3, 413, 0.0262157498993542],
 [2131, 444, 0.999868937924345],
 [2130, 487, 0.329325712658943],
 [2130, 410, 0.093541810615345],
 [2130, 78, 0.572072874980767],
 [2129, 9, 0.865598777104055],
 [2129, 372, 0.0989323896450954],
 [2129, 402, 0.0354688332508493],
 [157, 94, 0.902611552679438],
 [157, 286, 0.0340671553770332],
 [157, 200, 0.0186782260549867],
 [157, 414, 0.0109110112103501],
 [157, 499, 0.0158815907979228],
 [2113, 481, 0.208497799983058],
 [2113, 49, 0.790442352476713],
 [2108, 467, 0.0245741527285221],
 [2108, 466, 0.119040965097214],
 [2108, 499, 0.0129288241216356],
 [2108, 475, 0.127179476372808],
 [2108, 390, 0.0181104692919774],
 [2108, 362, 0.0153189946014242],
 [2108, 495, 0.340617342946671],
 [2108, 433, 0.299321540028514],
 [2108, 154, 0.0210596684167106],
 [2108, 498, 0.0165321343596279],
 [2128, 466, 0.126873193256315],
 [2128, 479, 0.

# GET ORIGINAL PHI 

In [52]:
# get the features related to the instances
feature_instance = FeatureInstance.objects.filter(document_id__in=experiment_docs)
feature_instance_join = {}
for i in feature_instance:
    feature_instance_join.update({int(i.id):[int(unique_docs[i.document_id]), int(unique_words[i.feature_id])]})

In [53]:
feature_m2m_instance = FeatureMass2MotifInstance.objects.filter(mass2motif__experiment=experiment)
phi_list = []
for i in feature_m2m_instance:
    if i.featureinstance_id in feature_instance_join.keys():
        phi_list.append([feature_instance_join[int(i.featureinstance_id)][0], unique_topics[int(i.mass2motif_id)], feature_instance_join[int(i.featureinstance_id)][1],i.probability])

In [54]:
# you will find phi here - doc, topic, word, probability 
# CHECKED VERSUS: 
# select fi.document_id, fm2m.mass2motif_id, fi.feature_id, probability   
# from basicviz_featureinstance fi, basicviz_featuremass2motifinstance fm2m
# where fm2m.featureinstance_id = fi.id
len(phi_list)

265053

In [55]:
phi_original = []
for line in phi_list: 
    phi_original.append([line[0],line[2],line[1],line[3]])
phi_original_array = np.array(phi_original)

In [59]:
np.savetxt("phi_original_array.csv", phi_original_array, delimiter=",", fmt="%s")

In [60]:
np.save("phi_original_array", phi_original_array)

# E-STEP (has 9 steps)

## Step 0 - E-step variables

In [158]:
# alpha_vector is already mentioned above
# beta_matrix is created here from pivot_table_normalised
# K and W are from above for total unique topics, total unique words respectively 
# you need a corpus (created above)
corpus = corpus_dict
beta_matrix = pivot_table_normalised

## Step 1 - Initialise phi

In [163]:
# initialise the 3D matrix phi
phi_matrix={}
for doc in corpus: 
    d = int(doc)
    phi_matrix[d] = {}
    for word in corpus[doc]:
        w = int(word)
        phi_matrix[d][w]=np.zeros(K)

## Step 2 - initialise gamma 

In [164]:
#create a gamma matrix with rows as documents and columns as topics 
#this will later be transposed in order to create the phi matrix in the steps 3-9 below
#doc_total is the count of words per doc, and each gamma is alpha plus that

gamma_matrix=np.zeros((int(len(corpus)),int(K))) #3x500 shape
for doc in corpus:
    doc_total=0.0
    for word in corpus[doc]:
        doc_total += corpus[doc][word]
    gamma_matrix[int(doc),:] = alpha_vector + 1.0*(doc_total/K)

In [68]:
# gamma_matrix_normalised maybe

## Step 3 - 9: repeat until convergence loop

In [165]:
# initialise phi and do Blei's loop 
beta_matrix = pivot_table_normalised
eta = 0.1
n_words = int(len(unique_words))
temp_beta = np.zeros((K, n_words))
for i in range(10):    
    for doc in corpus:
        d = int(doc)
        doc_dict = corpus[doc]
        temp_gamma = np.zeros(K) + alpha_vector
        for word in doc_dict: #the word is actually column positioning so we do not need n^3 complexity 
            w = int(word)
            log_phi_matrix = np.log(beta_matrix[:,w]) + psi(gamma_matrix[d,:]).T
            log_phi_matrix = np.exp(log_phi_matrix - log_phi_matrix.max())
            phi_matrix[d][w] = log_phi_matrix/log_phi_matrix.sum()
            temp_gamma += phi_matrix[d][w]*corpus[doc][word]
            temp_beta[:,w] += phi_matrix[d][w] * corpus[doc][word]
        gamma_matrix[d,:] += temp_gamma[0,]
        pos = np.where(gamma_matrix[d,:]<SMALL_NUMBER)[0]
        gamma_matrix[d,pos] = SMALL_NUMBER
    temp_beta += eta
    temp_beta /= temp_beta.sum(axis=1)[:,None]
    total_difference = (np.abs(temp_beta - beta_matrix)).sum()
#     beta_matrix = temp_beta
    print(i)
    print(total_difference)

0
177.4996468371373
1
166.59822947328217
2
160.41717027823933
3
158.15715918808183
4
157.0362289730486
5
156.365606089688
6
155.91949801424525
7
155.60585527105133
8
155.37615283864406
9
155.20730771043216


In [ ]:
phi_matrix

# COMPARISON (phi gamma original vs calculated) 

## Gamma comparison / actually Theta comparison 

## Phi comparison

In [88]:
len(unique_words)

27923

In [72]:
#check phi_matrix vs phi_list 
#first bring to same format

phi_calculated = []
for line in phi_list: 
    phi_calculated.append([line[0],line[2],line[1],phi_matrix[line[0]][line[2]][line[1]]])

phi_calculated_array = np.array(phi_calculated)
np.savetxt("phi_calculated_array.csv", phi_calculated_array, delimiter=",", fmt="%s")          
np.save("phi_calculated_array", phi_calculated_array)       